In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps
from FinMind.data import DataLoader
import helper
api = DataLoader()

/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def print_result(stats):
  stats = stats.loc[[
      'Start',
      'End',
      'Equity Final [$]',      # 最終資產淨值
      'Return [%]',            # 總報酬率
      'Buy & Hold Return [%]', # 買入並持有報酬率（基準）
      'Max. Drawdown [%]',     # 最大回撤
      '# Trades',
      'Win Rate [%]',          # 勝率
      'SQN',                    # 系統品質數字
      '_trades',
      '_strategy'
    ]]
  print('------------------回測表現-----------------')
  print(stats)
  print('------------------交易紀錄-----------------')
  trades = stats['_trades']
  for _, trade in trades.iterrows():
      print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
      print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

In [17]:
today = datetime.today() #datetime.strptime('2025-04-30', "%Y-%m-%d")
start_date = (today - relativedelta(years=2)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")
print('start date', start_date)
print('end date', end_date)

ticker = '2609'
df = api.taiwan_stock_daily(
    stock_id=ticker,
    start_date=start_date,
    end_date=end_date
)
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df = df.rename(columns={'max': 'High', 'min': 'Low', 'open': 'Open', 'close': 'Close', 'Trading_Volume': 'Volume'})
df

2025-12-07 18:03:40.293 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockPrice, data_id: 2609


start date 2023-12-07
end date 2025-12-07


,stock_id,Volume,Trading_money,Open,High,Low,Close,spread,Trading_turnover
date,,,,,,,,,
2023-12-07,2609,15228945,670567598,44.70,44.75,43.65,43.7,-0.95,8591
2023-12-08,2609,13164539,577298817,43.75,44.30,43.55,44.0,0.30,6330
2023-12-11,2609,16725517,740425368,44.45,44.85,43.85,44.0,0.00,7922
2023-12-12,2609,33867861,1506474335,44.20,45.30,44.00,44.0,0.00,15182
2023-12-13,2609,21555739,939649686,44.10,44.30,43.20,43.2,-0.80,12097
...,...,...,...,...,...,...,...,...,...
2025-12-01,2609,9570503,500459684,51.80,52.70,51.80,52.4,0.40,5662
2025-12-02,2609,12505765,653600746,52.20,52.80,52.00,52.0,-0.40,7047
2025-12-03,2609,7340873,382030284,52.20,52.40,51.80,51.9,-0.10,4928


In [18]:
# helper.EMA_KD.n1 = 20
# helper.EMA_KD.n2 = 60
bt = Backtest(df, helper.EMA_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[20, 50, 60, 100, 120],
  kd=85,
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/20 [00:00<?, ?it/s]/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=88: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=177: Broker canceled the r

------------------回測表現-----------------
Start                                                  2023-12-07 00:00:00
End                                                    2025-12-05 00:00:00
Equity Final [$]                                             185926.213892
Return [%]                                                       85.926214
Buy & Hold Return [%]                                             4.868154
Max. Drawdown [%]                                               -22.287018
# Trades                                                                 3
Win Rate [%]                                                         100.0
SQN                                                               3.394873
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                         EMA_KD(n1=5,n2=20,kd=85)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-03-01 00:00:00, Sell date: 2024-05-17 00:00:00
Buy price: 49.5, Se

In [11]:
# helper.EMA_VWAP_KD.n1 = 10
# helper.EMA_VWAP_KD.n2 = 50
bt = Backtest(df, helper.EMA_VWAP_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.run:   0%|          | 0/716 [00:00<?, ?bar/s]

/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=417: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=429: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=457: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=469: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=500: Broker canceled the

------------------回測表現-----------------
Start                                                  2022-12-07 00:00:00
End                                                    2025-12-05 00:00:00
Equity Final [$]                                             138493.889718
Return [%]                                                        38.49389
Buy & Hold Return [%]                                             4.593176
Max. Drawdown [%]                                               -26.906764
# Trades                                                                 3
Win Rate [%]                                                     66.666667
SQN                                                               1.886872
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                         EMA_VWAP_KD(n1=50,n2=10)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-07-22 00:00:00, Sell date: 2024-09-25 00:00:00
Buy price: 69.6, Se

In [5]:
# helper.SmaCross.n1 = 10
# helper.SmaCross.n2 = 100
bt = Backtest(df, helper.SmaCross, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  maximize='Equity Final [$]'
)
# stats = bt.run()
print_result(stats)

d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close


d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:168: RuntimeWarning: divide by zero encountered in divide
  x = value / self._data.Close
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=52: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=65: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=140: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=184: Brok

------------------回測表現-----------------
Start                                                  2023-11-13 00:00:00
End                                                    2025-11-11 00:00:00
Equity Final [$]                                             273178.203113
Return [%]                                                      173.178203
Buy & Hold Return [%]                                           142.857143
Max. Drawdown [%]                                               -99.974269
# Trades                                                                 2
Win Rate [%]                                                         100.0
SQN                                                               1.302876
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                             SmaCross(n1=5,n2=60)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-02-21 00:00:00, Sell date: 2024-08-06 00:00:00
Buy price: 103.5, S

In [22]:
bt = Backtest(df, helper.SMA_KD, cash=100_000, commission=.001425)
stats = bt.optimize(
  n1=[5, 10, 20, 50],
  n2=[10, 20, 50, 60, 100, 120],
  kd=85,
  maximize='Equity Final [$]'
)
print_result(stats)

/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/24 [00:00<?, ?it/s]/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=48: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=78: Broker canceled the 

------------------回測表現-----------------
Start                                                  2023-12-07 00:00:00
End                                                    2025-12-05 00:00:00
Equity Final [$]                                              206340.47649
Return [%]                                                      106.340476
Buy & Hold Return [%]                                            -9.773124
Max. Drawdown [%]                                               -22.283816
# Trades                                                                 3
Win Rate [%]                                                         100.0
SQN                                                               6.099174
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                        SMA_KD(n1=10,n2=20,kd=85)
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-03-04 00:00:00, Sell date: 2024-05-17 00:00:00
Buy price: 48.8, Se

/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1545: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = self.run(**dict(zip(heatmap.index.names, best_params)))


In [19]:
helper.BOLL_KD30.buy_strategy = helper.BuyStrategy.BOLL_KD30
bt = Backtest(df, helper.BOLL_KD30, cash=100_000, commission=.001425)
stats = bt.run()
print_result(stats)

Backtest.run:   0%|          | 0/465 [00:00<?, ?bar/s]/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=66: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=141: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=142: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:966: UserWarning: time=143: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/kaorinoie/Desktop/stock_trade/.venv/lib/python3.11/site-packages/backtesting/backtesting

------------------回測表現-----------------
Start                                                  2023-12-07 00:00:00
End                                                    2025-12-05 00:00:00
Equity Final [$]                                              82432.122266
Return [%]                                                      -17.567878
Buy & Hold Return [%]                                            -9.773124
Max. Drawdown [%]                                               -32.673098
# Trades                                                                 3
Win Rate [%]                                                     66.666667
SQN                                                              -0.753148
_trades                     Size  EntryBar  ExitBar  EntryPrice  ExitPr...
_strategy                                                        BOLL_KD30
dtype: object
------------------交易紀錄-----------------
Buy date: 2024-03-20 00:00:00, Sell date: 2024-04-25 00:00:00
Buy price: 45.45, S